In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import sentencepiece as spm


import string
import re
import random
import numpy as np
import os

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import sentencepiece as spm

from word2vector import Vocab
from EncoderRNN import EncoderRNN
from Attention import Attn
from DecoderRNN import DecoderRNN
from Data_reading import input_data
from Pretrained_embedding import pre_embedding
import Train


device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device)

cuda:2


In [2]:
sp_user = spm.SentencePieceProcessor()
sp_user.load('m.model')


True

In [3]:
MAX_LEN = 40
src_train, src_train_lens, tgt_train, tgt_train_lens, src_val, src_val_lens, tgt_val, tgt_val_lens =  input_data(sp_user, MAX_LEN)

src_train (98634, 40)
src_train_lens (98634,)
tgt_train (98634, 40)
tgt_train_lens (98634,)
src_val (4962, 40)
src_val_lens (4962,)
tgt_val (4962, 40)
tgt_val_lens (4962,)


In [4]:
# import matplotlib.pyplot as plt

In [5]:
# plt.hist(src_train_lens, np.max(src_train_lens))

In [6]:
# plt.hist(tgt_train_lens[tgt_train_lens<100], 100)

In [7]:
# plt.hist(src_val_lens, np.max(src_val_lens))

In [8]:
# plt.hist(tgt_val_lens, np.max(tgt_val_lens))

In [9]:
test = np.arange(5000)
formal_train_tensors = torch.LongTensor(src_train[test]).to(device)
formal_train_lens_tensors = torch.LongTensor(src_train_lens[test]).to(device)
informal_train_tensors = torch.LongTensor(tgt_train[test]).to(device)
informal_train_lens_tensors = torch.LongTensor(tgt_train_lens[test]).to(device)

test_1= np.arange(1000)
formal_val_tensors = torch.LongTensor(src_val[test_1]).to(device)
formal_val_lens_tensors = torch.LongTensor(src_val_lens[test_1]).to(device)
informal_val_tensors = torch.LongTensor(tgt_val[test_1]).to(device)
informal_val_lens_tensors = torch.LongTensor(tgt_val_lens[test_1]).to(device)

# formal_train_tensors = torch.LongTensor(src_train).to(device)
# formal_train_lens_tensors = torch.LongTensor(src_train_lens).to(device)
# informal_train_tensors = torch.LongTensor(tgt_train).to(device)
# informal_train_lens_tensors = torch.LongTensor(tgt_train_lens).to(device)

# formal_val_tensors = torch.LongTensor(src_val).to(device)
# formal_val_lens_tensors = torch.LongTensor(src_val_lens).to(device)
# informal_val_tensors = torch.LongTensor(tgt_val).to(device)
# informal_val_lens_tensors = torch.LongTensor(tgt_val_lens).to(device)

In [10]:
embedding_dim = 100
num_layer = 4
dropout = 0.5
hidden_dim = 128
batch_size = 64
learning_rate=0.001
vocab_size = sp_user.get_piece_size() #len(Glove.word2index)
#embedding=None
attn = 'general'
style_1 = sp_user.piece_to_id('<informal>')
style_2 = sp_user.piece_to_id('<formal>')

embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)

encoder = EncoderRNN(embedding=embedding, vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim, device=device ,
                     num_layer=num_layer, dropout=dropout).to(device)

decoder = DecoderRNN(embedding=embedding, attn_model=attn, vocab_size=vocab_size, embedding_dim=embedding_dim, 
                     hidden_dim=hidden_dim, device=device , num_layer=num_layer, dropout=dropout).to(device)

encoder_optimizer = optim.Adam(encoder.parameters(),lr=learning_rate)

decoder_optimizer = optim.Adam(decoder.parameters(),lr=learning_rate)

In [ ]:
output_seq, plot_train_loss, plot_val_loss = Train.train(formal_train_tensors, formal_train_lens_tensors, informal_train_tensors, informal_train_lens_tensors,
            formal_val_tensors, formal_val_lens_tensors, informal_val_tensors, informal_val_lens_tensors,
            style_1=style_1, style_2=style_2,
            encoder=encoder, decoder=decoder, encoder_optimizer=encoder_optimizer, decoder_optimizer=decoder_optimizer, 
            epoches=3, batch_size=batch_size, print_every=1, plot_every=1, sp_user=sp_user)

0.33333333333333337
iter 1
4992/5000, loss:16.686233520507812
Evalidation: 960/1000, loss:15.162066650390624
women are acquisitive . give them both and they will say , " it is not working well enough , darling ."
i i i i
i i i i
women are greedy , give them the both , and they will say , it isnot working good enough , honey
i i i i
i i i i

0m 45s (- 1m 30s) train loss: 16.046.  val loss: 13.735.
val loss decreases from 100.000 to 13.735, save models

-0.33333333333333326
iter 2
4992/5000, loss:17.27268409729004
Evalidation: 960/1000, loss:14.4638916015625
women are acquisitive . give them both and they will say , " it is not working well enough , darling ."
i i i i i i i , , , , , , , , , ,
i i i i i i i , , , , , , , , , ,
women are greedy , give them the both , and they will say , it isnot working good enough , honey
i i i i i i i , , , , , , , , , ,
i i i i i i i , , , , , , , , , ,

1m 31s (- 0m 45s) train loss: 12.111.  val loss: 13.231.
val loss decreases from 13.735 to 13.231, 

In [ ]:
# output_seq[0][0]

In [ ]:
# for i in range(6):
#     print(sp_user.decode_ids(output_seq[5][i].tolist()))

In [ ]:
with open('./saved_models/embedding', 'w') as f:
    print(embedding.weight)


In [ ]:
!jupyter nbconvert --to script unsupervised.ipynb